# para achar o race.csv e para ver as colunas e tipo delas

In [0]:
 #display(dbutils.fs.ls("dbfs:/mnt/formula1/bronze/race_raw.delta"))

# Leitura do arquivo Delta
df = spark.read.format("delta").load("dbfs:/mnt/formula1/bronze/race_raw.delta")

# Exibir o DataFrame para verificar o conteúdo
display(df)

##para renomeiar as duas colunas

In [0]:
%python
# Leitura do arquivo Delta
df = spark.read.format("delta").load("dbfs:/mnt/formula1/bronze/race_raw.delta")

# Renomear as colunas
df = df.withColumnRenamed("raceId", "Race_id")
df = df.withColumnRenamed("circuitId", "Circuit_id")

# Salvar o DataFrame de volta como Delta em um diretório
output_directory = "dbfs:/mnt/formula1/bronze/race_raw.delta"
df.write.format("delta").mode('overwrite').save("dbfs:/mnt/formula1/prata/race_raw.delta")

# Exibir o DataFrame para verificar a alteração
display(df)

## para deletar url

In [0]:
%python
# Verificar se o caminho existe
try:
    dbutils.fs.ls('dbfs:/mnt/formula1/bronze/race_raw.delta')
    # Se o caminho existir, carregar os dados
    df = spark.read.format('delta').load('dbfs:/mnt/formula1/bronze/race_raw.delta')
    
    # Excluir a coluna 'url'
    df = df.drop("url")

    # Exibir o DataFrame para verificar a alteração
    display(df)
except Exception as e:
    print(f"Erro: {e}")

## para criar uma nova coluna para receber o tempo alterado

In [0]:
%python
# Verificar se o caminho existe
try:
    dbutils.fs.ls('dbfs:/mnt/formula1/bronze/race_raw.delta')
    # Se o caminho existir, carregar os dados
    df = spark.read.format('delta').load('dbfs:/mnt/formula1/bronze/race_raw.delta')
    
    from pyspark.sql.functions import lit, concat, to_timestamp, date_format

    # Supondo que as colunas 'time' e 'date' existam no DataFrame
    # Concatenar as colunas 'date' e 'time' e convertê-las para o formato 'yyyy-MM-dd HH:mm:ss'
    df = df.withColumn("coluna do tempo", date_format(to_timestamp(concat(df["date"], lit(" "), df["time"])), "yyyy-MM-dd HH:mm:ss"))

    # Exibir o DataFrame para verificar a alteração
    display(df)
except Exception as e:
    print(f"Erro: {e}")

In [0]:
%python
# Verificar se o caminho existe
try:
    dbutils.fs.ls('dbfs:/mnt/formula1/prata/race_raw.delta')
    # Se o caminho existir, carregar os dados
    df = spark.read.format('delta').load('dbfs:/mnt/formula1/prata/race_raw.delta')
    display(df)
except Exception as e:
    print(f"Erro: {e}")